In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow import keras
from tensorflow.keras import layers as L
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import numpy as np 

In [2]:
df_models = pd.read_csv("данные для моделей.csv")
print("DataFrame 'данные для моделей.csv' успешно загружен.")

print("\n--- Классификация: превышает ли значение SI пороговое значение 8 (log_SI > log10(8)) ---")

# 1. Определение целевой переменной для SI > 8
target_si_above_8_class = 'si_above_8' # Новая целевая колонка

# Порог 8 в логарифмированном масштабе (по основанию 10)
threshold_si_log10_8 = np.log10(8)
print(f"Пороговое значение SI=8 в логарифмическом масштабе (log10(8)): {threshold_si_log10_8:.4f}")

# Создаем бинарный столбец: 1, если log_SI > log10(8), иначе 0
df_models[target_si_above_8_class] = (df_models['log_SI'] > threshold_si_log10_8).astype(int)

print(f"Распределение классов для {target_si_above_8_class}:\n{df_models[target_si_above_8_class].value_counts()}")

# Признаки (X) и целевая переменная (y)
# Исключаем исходные логарифмированные целевые переменные и только что созданную бинарную целевую переменную.
X = df_models.drop(columns=['log_IC50, mM', 'log_CC50, mM', 'log_SI', target_si_above_8_class])
y = df_models[target_si_above_8_class]

# Разделение на обучающую и тестовую выборки
X_train_si_8_class, X_test_si_8_class, y_train_si_8_class, y_test_si_8_class = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y # stratify=y для сохранения пропорций классов
)

print(f"\nРазмер обучающей выборки (X_train_si_8_class): {X_train_si_8_class.shape}")
print(f"Размер тестовой выборки (X_test_si_8_class): {X_test_si_8_class.shape}")
print(f"Распределение классов в обучающей выборке:\n{y_train_si_8_class.value_counts(normalize=True)}")
print(f"Распределение классов в тестовой выборке:\n{y_test_si_8_class.value_counts(normalize=True)}")

DataFrame 'данные для моделей.csv' успешно загружен.

--- Классификация: превышает ли значение SI пороговое значение 8 (log_SI > log10(8)) ---
Пороговое значение SI=8 в логарифмическом масштабе (log10(8)): 0.9031
Распределение классов для si_above_8:
si_above_8
0    623
1    344
Name: count, dtype: int64

Размер обучающей выборки (X_train_si_8_class): (773, 68)
Размер тестовой выборки (X_test_si_8_class): (194, 68)
Распределение классов в обучающей выборке:
si_above_8
0    0.644243
1    0.355757
Name: proportion, dtype: float64
Распределение классов в тестовой выборке:
si_above_8
0    0.64433
1    0.35567
Name: proportion, dtype: float64


Видим явный дисбаланс классов. Будем использовать балансировку классов.

In [4]:
# --- Расчет весов классов для моделей ---
# Это будет использоваться для LGBM, XGBoost, CatBoost и Keras
# ratio = count(negative_class) / count(positive_class)
neg_count = y_train_si_8_class.value_counts()[0]
pos_count = y_train_si_8_class.value_counts()[1]
scale_pos_weight_value = neg_count / pos_count
print(f"\nРассчитанное значение scale_pos_weight: {scale_pos_weight_value:.4f}")


Рассчитанное значение scale_pos_weight: 1.8109


In [5]:
# Метрики для GridSearchCV
scoring_metrics_class = {
    'Accuracy': 'accuracy',
    'F1': 'f1',
    'ROC_AUC': 'roc_auc'
}

In [6]:
# Модель 1: Логистическая Регрессия 
print("\n===== Логистическая Регрессия для SI > 8 (классификация) =====")
pipeline_lr_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear',
                                      class_weight='balanced')) # балансировка классов
])

param_grid_lr_class_si_8 = {
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2']
}

grid_search_lr_class_si_8 = GridSearchCV(
    pipeline_lr_class_si_8,
    param_grid_lr_class_si_8,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_lr_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для Логистической Регрессии:", grid_search_lr_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lr_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_lr_class_si_8 = grid_search_lr_class_si_8.predict(X_test_si_8_class)
y_proba_lr_class_si_8 = grid_search_lr_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_lr_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_lr_class_si_8)
f1_lr_class_si_8 = f1_score(y_test_si_8_class, y_pred_lr_class_si_8)
roc_auc_lr_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_lr_class_si_8)

print("\nМетрики на тестовой выборке (Логистическая Регрессия):")
print(f"Accuracy: {accuracy_lr_class_si_8:.4f}")
print(f"F1-score: {f1_lr_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_lr_class_si_8:.4f}")
print("\nClassification Report (Логистическая Регрессия):")
print(classification_report(y_test_si_8_class, y_pred_lr_class_si_8))


===== Логистическая Регрессия для SI > 8 (классификация) =====
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшие параметры для Логистической Регрессии: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC_AUC на кросс-валидации: 0.7034995408631772

Метрики на тестовой выборке (Логистическая Регрессия):
Accuracy: 0.6649
F1-score: 0.5806
ROC AUC: 0.6991

Classification Report (Логистическая Регрессия):
              precision    recall  f1-score   support

           0       0.78      0.67      0.72       125
           1       0.52      0.65      0.58        69

    accuracy                           0.66       194
   macro avg       0.65      0.66      0.65       194
weighted avg       0.69      0.66      0.67       194



In [7]:
# Модель 2: Random Forest Классификатор 
print("\n===== Random Forest Классификатор для SI > 8 (классификация) =====")
pipeline_rf_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42,
                                          class_weight='balanced')) #  балансировка классов
])

param_grid_rf_class_si_8 = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_features': [0.6, 0.8, 1.0],
    'classifier__min_samples_leaf': [5, 10]
}

grid_search_rf_class_si_8 = GridSearchCV(
    pipeline_rf_class_si_8,
    param_grid_rf_class_si_8,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_rf_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для Random Forest:", grid_search_rf_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_rf_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_rf_class_si_8 = grid_search_rf_class_si_8.predict(X_test_si_8_class)
y_proba_rf_class_si_8 = grid_search_rf_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_rf_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_rf_class_si_8)
f1_rf_class_si_8 = f1_score(y_test_si_8_class, y_pred_rf_class_si_8)
roc_auc_rf_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_rf_class_si_8)

print("\nМетрики на тестовой выборке (Random Forest):")
print(f"Accuracy: {accuracy_rf_class_si_8:.4f}")
print(f"F1-score: {f1_rf_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_rf_class_si_8:.4f}")
print("\nClassification Report (Random Forest):")
print(classification_report(y_test_si_8_class, y_pred_rf_class_si_8))


===== Random Forest Классификатор для SI > 8 (классификация) =====
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Лучшие параметры для Random Forest: {'classifier__max_features': 0.6, 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 100}
Лучший ROC_AUC на кросс-валидации: 0.7463399449035812

Метрики на тестовой выборке (Random Forest):
Accuracy: 0.6959
F1-score: 0.5986
ROC AUC: 0.7212

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.78      0.73      0.76       125
           1       0.56      0.64      0.60        69

    accuracy                           0.70       194
   macro avg       0.67      0.68      0.68       194
weighted avg       0.71      0.70      0.70       194



In [8]:
#  Модель 3: LightGBM Классификатор
print("\n===== LightGBM Классификатор для SI > 8 (классификация) =====")
pipeline_lgbm_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(random_state=42, verbose=-1,
                                  scale_pos_weight=scale_pos_weight_value)) #  балансировка классов
])

param_grid_lgbm_class_si_8 = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__num_leaves': [20, 31],
    'classifier__reg_alpha': [0.1, 0.5],
    'classifier__reg_lambda': [0.1, 0.5]
}

grid_search_lgbm_class_si_8 = GridSearchCV(
    pipeline_lgbm_class_si_8,
    param_grid_lgbm_class_si_8,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_lgbm_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для LightGBM:", grid_search_lgbm_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lgbm_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_lgbm_class_si_8 = grid_search_lgbm_class_si_8.predict(X_test_si_8_class)
y_proba_lgbm_class_si_8 = grid_search_lgbm_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_lgbm_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_lgbm_class_si_8)
f1_lgbm_class_si_8 = f1_score(y_test_si_8_class, y_pred_lgbm_class_si_8)
roc_auc_lgbm_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_lgbm_class_si_8)

print("\nМетрики на тестовой выборке (LightGBM):")
print(f"Accuracy: {accuracy_lgbm_class_si_8:.4f}")
print(f"F1-score: {f1_lgbm_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_lgbm_class_si_8:.4f}")
print("\nClassification Report (LightGBM):")
print(classification_report(y_test_si_8_class, y_pred_lgbm_class_si_8))


===== LightGBM Классификатор для SI > 8 (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Лучшие параметры для LightGBM: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100, 'classifier__num_leaves': 31, 'classifier__reg_alpha': 0.5, 'classifier__reg_lambda': 0.5}
Лучший ROC_AUC на кросс-валидации: 0.7440582185491276

Метрики на тестовой выборке (LightGBM):
Accuracy: 0.6907
F1-score: 0.5890
ROC AUC: 0.7177

Classification Report (LightGBM):
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       125
           1       0.56      0.62      0.59        69

    accuracy                           0.69       194
   macro avg       0.67      0.68      0.67       194
weighted avg       0.70      0.69      0.69       194



In [9]:
# Модель 4: XGBoost Классификатор
print("\n===== XGBoost Классификатор для SI > 8 (классификация) =====")
pipeline_xgb_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss',
                                 scale_pos_weight=scale_pos_weight_value)) #  балансировка классов
])

param_grid_xgb_class_si_8 = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__subsample': [0.7, 1.0],
    'classifier__colsample_bytree': [0.7, 1.0]
}

grid_search_xgb_class_si_8 = GridSearchCV(
    pipeline_xgb_class_si_8,
    param_grid_xgb_class_si_8,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для XGBoost:", grid_search_xgb_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_xgb_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_xgb_class_si_8 = grid_search_xgb_class_si_8.predict(X_test_si_8_class)
y_proba_xgb_class_si_8 = grid_search_xgb_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_xgb_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_xgb_class_si_8)
f1_xgb_class_si_8 = f1_score(y_test_si_8_class, y_pred_xgb_class_si_8)
roc_auc_xgb_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_xgb_class_si_8)

print("\nМетрики на тестовой выборке (XGBoost):")
print(f"Accuracy: {accuracy_xgb_class_si_8:.4f}")
print(f"F1-score: {f1_xgb_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_xgb_class_si_8:.4f}")
print("\nClassification Report (XGBoost):")
print(classification_report(y_test_si_8_class, y_pred_xgb_class_si_8))


===== XGBoost Классификатор для SI > 8 (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits


C:\Users\admin\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:53:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Лучшие параметры для XGBoost: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC_AUC на кросс-валидации: 0.7490370982552801

Метрики на тестовой выборке (XGBoost):
Accuracy: 0.6753
F1-score: 0.5882
ROC AUC: 0.7201

Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.78      0.69      0.73       125
           1       0.54      0.65      0.59        69

    accuracy                           0.68       194
   macro avg       0.66      0.67      0.66       194
weighted avg       0.69      0.68      0.68       194



In [10]:
# Модель 5: CatBoost Классификатор 
print("\n===== CatBoost Классификатор для SI > 8 (классификация) =====")
pipeline_cat_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', CatBoostClassifier(random_state=42, verbose=0,
                                      auto_class_weights='Balanced')) #  балансировка классов
])

param_grid_cat_class_si_8 = {
    'classifier__iterations': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__depth': [4, 6],
    'classifier__l2_leaf_reg': [1, 3]
}

grid_search_cat_class_si_8 = GridSearchCV(
    pipeline_cat_class_si_8,
    param_grid_cat_class_si_8,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_cat_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для CatBoost:", grid_search_cat_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_cat_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_cat_class_si_8 = grid_search_cat_class_si_8.predict(X_test_si_8_class)
y_proba_cat_class_si_8 = grid_search_cat_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_cat_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_cat_class_si_8)
f1_cat_class_si_8 = f1_score(y_test_si_8_class, y_pred_cat_class_si_8)
roc_auc_cat_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_cat_class_si_8)

print("\nМетрики на тестовой выборке (CatBoost):")
print(f"Accuracy: {accuracy_cat_class_si_8:.4f}")
print(f"F1-score: {f1_cat_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_cat_class_si_8:.4f}")
print("\nClassification Report (CatBoost):")
print(classification_report(y_test_si_8_class, y_pred_cat_class_si_8))


===== CatBoost Классификатор для SI > 8 (классификация) =====
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Лучшие параметры для CatBoost: {'classifier__depth': 6, 'classifier__iterations': 100, 'classifier__l2_leaf_reg': 3, 'classifier__learning_rate': 0.05}
Лучший ROC_AUC на кросс-валидации: 0.747685583103765

Метрики на тестовой выборке (CatBoost):
Accuracy: 0.6701
F1-score: 0.5789
ROC AUC: 0.7007

Classification Report (CatBoost):
              precision    recall  f1-score   support

           0       0.77      0.69      0.73       125
           1       0.53      0.64      0.58        69

    accuracy                           0.67       194
   macro avg       0.65      0.66      0.65       194
weighted avg       0.69      0.67      0.68       194



In [11]:
# Модель 6: Простая Нейронная Сеть (Keras Sequential) 
print("\n===== Простая Нейронная Сеть для SI > 8 (классификация) =====")

# Функция для создания Keras-модели (используем ту же, что и раньше)
def build_nn_model_class(meta, hidden_layers=1, neurons=32, activation='relu',
                         optimizer='adam', learning_rate=0.001):
    n_features = meta["n_features_in_"]
    
    model = keras.Sequential()
    model.add(L.Input(shape=(n_features,)))
    
    for _ in range(hidden_layers):
        model.add(L.Dense(neurons, activation=activation))
        
    model.add(L.Dense(1, activation='sigmoid')) # Sigmoid для бинарной классификации
    
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

#  class_weight непосредственно в метод fit() 
# Scikeras (обертка KerasClassifier) позволяет передавать class_weight в KerasClassifier.

# Сначала, создадим словарь весов классов
class_weights_nn = {0: 1.0, 1: scale_pos_weight_value}
print(f"Веса классов для Нейронной Сети: {class_weights_nn}")

pipeline_nn_class_si_8 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', KerasClassifier(
        model=build_nn_model_class,
        hidden_layers=1,
        neurons=32,
        activation='relu',
        optimizer='adam',
        learning_rate=0.001,
        batch_size=32,
        epochs=50,
        verbose=0,
        random_state=42,
        loss='binary_crossentropy',
        class_weight=class_weights_nn #  балансировка классов
    ))
])

param_grid_nn_class_si_8 = {
    'classifier__hidden_layers': [1, 2],
    'classifier__neurons': [32, 64],
    'classifier__activation': ['relu'],
    'classifier__optimizer': ['adam'],
    'classifier__learning_rate': [0.001, 0.01],
    'classifier__batch_size': [32, 64],
    'classifier__epochs': [50, 100]
}

grid_search_nn_class_si_8 = GridSearchCV(
    pipeline_nn_class_si_8,
    param_grid_nn_class_si_8,
    cv=3, # Уменьшаем CV для скорости NN
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_nn_class_si_8.fit(X_train_si_8_class, y_train_si_8_class)

print("Лучшие параметры для Нейронной Сети:", grid_search_nn_class_si_8.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_nn_class_si_8.best_score_)

# Оценка на тестовой выборке
y_pred_nn_class_si_8 = grid_search_nn_class_si_8.predict(X_test_si_8_class)
y_proba_nn_class_si_8 = grid_search_nn_class_si_8.predict_proba(X_test_si_8_class)[:, 1]

accuracy_nn_class_si_8 = accuracy_score(y_test_si_8_class, y_pred_nn_class_si_8)
f1_nn_class_si_8 = f1_score(y_test_si_8_class, y_pred_nn_class_si_8)
roc_auc_nn_class_si_8 = roc_auc_score(y_test_si_8_class, y_proba_nn_class_si_8)

print("\nМетрики на тестовой выборке (Нейронная Сеть):")
print(f"Accuracy: {accuracy_nn_class_si_8:.4f}")
print(f"F1-score: {f1_nn_class_si_8:.4f}")
print(f"ROC AUC: {roc_auc_nn_class_si_8:.4f}")
print("\nClassification Report (Нейронная Сеть):")
print(classification_report(y_test_si_8_class, y_pred_nn_class_si_8))


===== Простая Нейронная Сеть для SI > 8 (классификация) =====
Веса классов для Нейронной Сети: {0: 1.0, 1: 1.8109090909090908}
Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\admin\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Лучшие параметры для Нейронной Сети: {'classifier__activation': 'relu', 'classifier__batch_size': 64, 'classifier__epochs': 50, 'classifier__hidden_layers': 2, 'classifier__learning_rate': 0.001, 'classifier__neurons': 32, 'classifier__optimizer': 'adam'}
Лучший ROC_AUC на кросс-валидации: 0.7345454440792314

Метрики на тестовой выборке (Нейронная Сеть):
Accuracy: 0.6186
F1-score: 0.5195
ROC AUC: 0.6635

Classification Report (Нейронная Сеть):
              precision    recall  f1-score   support

           0       0.73      0.64      0.68       125
           1       0.47      0.58      0.52        69

    accuracy                           0.62       194
   macro avg       0.60      0.61      0.60       194
weighted avg       0.64      0.62      0.63       194



Распределение классов для si_above_8 является несбалансированным: класс 0 (SI <= 8) содержит 623 образца, а класс 1 (SI > 8) — 344 образца. В обучающей и тестовой выборках это соотношение сохраняется (примерно 64.4% для класса 0 и 35.6% для класса 1). Для компенсации этого дисбаланса был рассчитан scale_pos_weight со значением 1.8109.

1. Логистическая Регрессия
Лучшие параметры: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC AUC на кросс-валидации: 0.7035
Метрики на тестовой выборке:
Accuracy: 0.6649
F1-score: 0.5806
ROC AUC: 0.6991
Classification Report:
Precision (класс 0 / класс 1): 0.78 / 0.52
Recall (класс 0 / класс 1): 0.67 / 0.65
F1-score (класс 0 / класс 1): 0.72 / 0.58
Логистическая регрессия показала ROC AUC 0.6991 на тестовой выборке, что является одним из самых высоких значений среди всех моделей для данной задачи. Accuracy составила 0.6649. Из отчёта по классификации видно, что F1-score для класса 1 (0.58) значительно ниже, чем для класса 0 (0.72). Это связано с низкой Precision для класса 1 (0.52), несмотря на относительно неплохой Recall (0.65). Модель склонна к ложным срабатываниям для класса 1.

2. Random Forest Классификатор
Лучшие параметры: {'classifier__max_features': 0.6, 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 100}
Лучший ROC AUC на кросс-валидации: 0.7463
Метрики на тестовой выборке:
Accuracy: 0.6959
F1-score: 0.5986
ROC AUC: 0.7212
Classification Report:
Precision (класс 0 / класс 1): 0.78 / 0.56
Recall (класс 0 / класс 1): 0.73 / 0.64
F1-score (класс 0 / класс 1): 0.76 / 0.60
Анализ: Random Forest показал лучшие результаты по ROC AUC (0.7212) и Accuracy (0.6959) на тестовой выборке. F1-score для класса 1 (0.60) немного выше, чем у логистической регрессии, но всё ещё остаётся невысоким из-за низкой Precision (0.56) для этого класса. Модель демонстрирует лучшее равновесие между Precision и Recall для класса 1 по сравнению с Логистической регрессией.

3. LightGBM Классификатор
Лучшие параметры: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100, 'classifier__num_leaves': 31, 'classifier__reg_alpha': 0.5, 'classifier__reg_lambda': 0.5}
Лучший ROC AUC на кросс-валидации: 0.7441
Метрики на тестовой выборке:
Accuracy: 0.6907
F1-score: 0.5890
ROC AUC: 0.7177
Classification Report:
Precision (класс 0 / класс 1): 0.78 / 0.56
Recall (класс 0 / класс 1): 0.73 / 0.62
F1-score (класс 0 / класс 1): 0.75 / 0.59
Анализ: LightGBM показал ROC AUC 0.7177 на тестовой выборке, близкий к Random Forest. Accuracy (0.6907) и F1-score (0.5890) также сопоставимы. Отчет по классификации демонстрирует схожую картину с Random Forest: высокая Precision для класса 0 (0.78) и относительно низкая Precision для класса 1 (0.56), указывающая на трудности в точной идентификации положительного класса.

4. XGBoost Классификатор
Лучшие параметры: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC AUC на кросс-валидации: 0.7490
Метрики на тестовой выборке:
Accuracy: 0.6753
F1-score: 0.5882
ROC AUC: 0.7201
Classification Report:
Precision (класс 0 / класс 1): 0.78 / 0.54
Recall (класс 0 / класс 1): 0.69 / 0.65
F1-score (класс 0 / класс 1): 0.73 / 0.59
Анализ: XGBoost достиг ROC AUC 0.7201 на тестовой выборке, что является одним из лучших результатов. Accuracy (0.6753) и F1-score (0.5882) также сравнимы с другими моделями. Наблюдается тот же паттерн: высокая Precision для класса 0 (0.78) и низкая для класса 1 (0.54), что приводит к умеренному F1-score для класса 1.

5. CatBoost Классификатор
Лучшие параметры: {'classifier__depth': 6, 'classifier__iterations': 100, 'classifier__l2_leaf_reg': 3, 'classifier__learning_rate': 0.05}
Лучший ROC AUC на кросс-валидации: 0.7477
Метрики на тестовой выборке:
Accuracy: 0.6701
F1-score: 0.5789
ROC AUC: 0.7007
Classification Report:
Precision (класс 0 / класс 1): 0.77 / 0.53
Recall (класс 0 / класс 1): 0.69 / 0.64
F1-score (класс 0 / класс 1): 0.73 / 0.58
Анализ: CatBoost показал ROC AUC 0.7007 на тестовой выборке, что является хорошим результатом. Accuracy (0.6701) и F1-score (0.5789) схожи с другими моделями. Отчет по классификации подтверждает общую тенденцию: низкая Precision для класса 1 (0.53) при относительно неплохом Recall (0.64), что приводит к невысокому F1-score для положительного класса.

6. Простая Нейронная Сеть
Лучшие параметры: {'classifier__activation': 'relu', 'classifier__batch_size': 64, 'classifier__epochs': 50, 'classifier__hidden_layers': 2, 'classifier__learning_rate': 0.001, 'classifier__neurons': 32, 'classifier__optimizer': 'adam'}
Лучший ROC AUC на кросс-валидации: 0.7345
Метрики на тестовой выборке:
Accuracy: 0.6186
F1-score: 0.5195
ROC AUC: 0.6635
Classification Report:
Precision (класс 0 / класс 1): 0.73 / 0.47
Recall (класс 0 / класс 1): 0.64 / 0.58
F1-score (класс 0 / класс 1): 0.68 / 0.52
Анализ: Простая нейронная сеть показала самые низкие результаты по всем метрикам на тестовой выборке среди всех моделей, включая ROC AUC 0.6635. Accuracy (0.6186) и F1-score (0.5195) также являются самыми низкими. Особенно заметна очень низкая Precision для класса 1 (0.47), что делает её наименее подходящей для этой задачи.

Общий вывод по классификации: SI > 8
Задача классификации SI выше порогового значения 8 оказалась самой сложной из всех классификационных задач, что подтверждается относительно низкими значениями всех метрик. ROC AUC для большинства моделей находится в диапазоне 0.66–0.72, а F1-score для класса 1 (положительного класса, SI > 8) остаётся низким, в пределах 0.52–0.60. Это указывает на значительные трудности в точном определении образцов с высоким SI.

Наилучшие результаты по ROC AUC на тестовой выборке показали Random Forest (0.7212) и XGBoost (0.7201), незначительно опережая Логистическую регрессию (0.6991) и LightGBM (0.7177). Однако, даже у лучших моделей, Precision для класса 1 остаётся на низком уровне (0.52-0.56), что приводит к большому количеству ложных положительных предсказаний.

Несбалансированность классов (класс 0 значительно больше класса 1) вероятно, играет роль в сложности этой задачи. Хотя scale_pos_weight был использован, это не полностью решило проблему низкой Precision для класса 1.

Вывод:
Для задачи классификации "превышает ли значение SI пороговое значение 8" ни одна из моделей не достигла высокого уровня производительности. Random Forest и XGBoost показали себя несколько лучше остальных по ROC AUC, но общая эффективность остаётся умеренной. Для значительного улучшения качества классификации в этой задаче потребуются более глубокая работа с признаками, возможно, их расширение, а также более специализированные методы для работы с несбалансированными данными или дальнейшая тонкая настройка моделей.

In [1]:
import sys
print(sys.version)
print(sys.executable)

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
C:\Users\admin\anaconda3\python.exe
